In [ ]:

# Financial Plan Summary Application
# ============================================================================
# INSTALLATION: Run this cell first to install all dependencies
# ============================================================================
!pip install langchain langchain-openai langchain-core langgraph openai plotly numpy


In [ ]:
# ============================================================================
# IMPORTS
# ============================================================================
import os
from typing import TypedDict, Annotated, Sequence, List, Dict, Any
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
import operator
from datetime import datetime
import json

# STEP 1: Set Your OpenAI API Key


In [ ]:
OPENAI_API_KEY = input("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# STEP 2: Define Shared Tools

This code block first prompts the user to enter their OpenAI API Key and then sets it as an environment variable. After that, it initializes the `ChatOpenAI` Large Language Model (LLM) with the specified model and temperature, which will be used by the agents throughout the application.

This code block first prompts the user to enter their OpenAI API Key and then sets it as an environment variable. After that, it initializes the `ChatOpenAI` Large Language Model (LLM) with the specified model and temperature, which will be used by the agents throughout the application.

This code block defines several helper functions, each representing a financial calculation tool. These tools are decorated with `@tool` from the `langchain` library, making them callable by the LLM-powered agents. The tools cover calculations for retirement needs, life insurance, education funds, estate tax, and wealth allocation.

In [ ]:
@tool
def calculate_retirement_needs(current_age: int, retirement_age: int,
                               annual_expenses: float, life_expectancy: int = 85) -> str:
    """Calculate estimated retirement fund needed based on age and expenses."""
    years_to_retirement = retirement_age - current_age
    years_in_retirement = life_expectancy - retirement_age

    # Simple calculation with 3% inflation
    inflation_rate = 0.03
    future_annual_expenses = annual_expenses * ((1 + inflation_rate) ** years_to_retirement)
    total_needed = future_annual_expenses * years_in_retirement

    return f"""Retirement Calculation:
- Years until retirement: {years_to_retirement}
- Years in retirement: {years_in_retirement}
- Future annual expenses (adjusted for inflation): ${future_annual_expenses:,.2f}
- Total retirement fund needed: ${total_needed:,.2f}
- Recommended monthly savings: ${(total_needed / (years_to_retirement * 12)):,.2f}"""

@tool
def calculate_life_insurance(annual_income: float, num_dependents: int,
                            outstanding_debts: float, savings: float) -> str:
    """Calculate recommended life insurance coverage."""
    # Rule of thumb: 10x annual income + debts - savings
    base_coverage = (annual_income * 10) + outstanding_debts - savings
    dependent_factor = 1 + (num_dependents * 0.2)
    recommended_coverage = base_coverage * dependent_factor

    return f"""Life Insurance Recommendation:
- Base coverage needed: ${base_coverage:,.2f}
- Adjustment for {num_dependents} dependent(s): {dependent_factor}x
- Recommended coverage: ${recommended_coverage:,.2f}
- Estimated monthly premium (term life): ${(recommended_coverage * 0.0005 / 12):,.2f}"""

@tool
def calculate_education_fund(num_children: int, children_ages: List[int],
                             cost_per_year: float = 30000) -> str:
    """Calculate education fund needed for children."""
    total_needed = 0
    details = []

    for i, age in enumerate(children_ages):
        years_until_college = max(0, 18 - age)
        years_in_college = 4

        # Inflation adjusted cost
        inflation_rate = 0.05
        future_cost = cost_per_year * ((1 + inflation_rate) ** years_until_college)
        child_total = future_cost * years_in_college
        total_needed += child_total

        details.append(f"Child {i+1} (age {age}): ${child_total:,.2f} needed in {years_until_college} years")

    return f"""Education Fund Calculation:
{chr(10).join(details)}
- Total education fund needed: ${total_needed:,.2f}
- Recommended monthly savings: ${(total_needed / (max([18 - age for age in children_ages if age < 18] + [1]) * 12)):,.2f}"""

@tool
def calculate_estate_tax(total_assets: float, state: str = "Federal") -> str:
    """Estimate potential estate taxes."""
    federal_exemption = 13610000  # 2024 exemption
    taxable_estate = max(0, total_assets - federal_exemption)
    estimated_tax = taxable_estate * 0.40  # Top federal rate

    return f"""Estate Tax Estimation:
- Total assets: ${total_assets:,.2f}
- Federal exemption: ${federal_exemption:,.2f}
- Taxable estate: ${taxable_estate:,.2f}
- Estimated federal estate tax: ${estimated_tax:,.2f}
- Recommendation: {'Estate planning strategies recommended' if taxable_estate > 0 else 'Below exemption threshold'}"""

@tool
def calculate_wealth_allocation(total_assets: float, age: int, risk_tolerance: str = "moderate") -> str:
    """Recommend asset allocation based on age and risk tolerance."""
    # Age-based stock allocation
    base_stock_pct = 100 - age

    # Adjust for risk tolerance
    if risk_tolerance.lower() == "aggressive":
        stock_pct = min(90, base_stock_pct + 10)
    elif risk_tolerance.lower() == "conservative":
        stock_pct = max(20, base_stock_pct - 20)
    else:
        stock_pct = base_stock_pct

    bond_pct = 100 - stock_pct

    stock_amount = total_assets * (stock_pct / 100)
    bond_amount = total_assets * (bond_pct / 100)

    return f"""Asset Allocation Recommendation:
- Risk Profile: {risk_tolerance.title()}
- Stocks/Equity: {stock_pct}% (${stock_amount:,.2f})
- Bonds/Fixed Income: {bond_pct}% (${bond_amount:,.2f})
- Rebalance: Quarterly or when allocation drifts 5%+"""

# STEP 3: Define State for Agent Graph
The AgentState in this notebook is a TypedDict that defines the structure of the state object passed between different agents in the financial planning application. It holds all the necessary information for the agents to process and build the financial plan. Here's a breakdown of its components:

* messages: A sequence of HumanMessage, AIMessage, or SystemMessage objects, representing the conversation history. It accumulates messages as the planning progresses.

* user_info: A dictionary containing all the personal and financial information provided by the user (e.g., age, income, savings, risk tolerance, number of dependents, etc.).

* selected_plans: A list of strings indicating which financial planning areas the user has chosen (e.g., 'Retirement Planning', 'Insurance Planning').

* plan_summaries: A dictionary where keys are the names of the planning areas and values are the generated summaries for each area. This is where the output of individual agents is stored.

* next_agent: A string indicating which agent should be executed next in a more complex graph flow, though in this specific setup, the OrchestratorAgent explicitly calls the other agents.

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[HumanMessage | AIMessage | SystemMessage], operator.add]
    user_info: Dict[str, Any]
    selected_plans: List[str]
    plan_summaries: Dict[str, str]
    next_agent: str

# STEP 4: Create Specialized Agents
Defines four specialized agent classes: RetirementAgent, InsuranceAgent, EstateAgent, and WealthAgent.

Each of these agents is designed to handle a specific financial planning domain:

* Initialization (__init__): Each agent is initialized with the llm (Large
Language Model) and a list of tools relevant to its domain. For example, RetirementAgent uses calculate_retirement_needs and calculate_wealth_allocation.
* Processing (process): This is the core method for each agent. It takes the current AgentState as input.
  * It constructs a detailed prompt for the LLM based on the user_info from the state.
  * It binds the agent's specific tools to the LLM.
  * It invokes the LLM with the prompt. The LLM then decides whether to call any of the bound tools.
  * If the LLM generates tool_calls, the agent executes these tools and uses their results to generate a more comprehensive summary.
  * Finally, it updates the plan_summaries dictionary in the AgentState with the generated summary for its respective planning area and adds a confirmation message to the messages list.

In [ ]:
class RetirementAgent:
    def __init__(self, llm):
        self.llm = llm
        self.tools = [calculate_retirement_needs, calculate_wealth_allocation]

    def process(self, state: AgentState) -> AgentState:
        user_info = state["user_info"]

        prompt = f"""You are a Retirement Planning Specialist. Analyze the user's information and create a comprehensive retirement plan.

User Information:
- Age: {user_info.get('age', 'Not provided')}
- Retirement Age: {user_info.get('retirement_age', 'Not provided')}
- Annual Income: ${user_info.get('annual_income', 0):,.2f}
- Current Savings: ${user_info.get('savings', 0):,.2f}
- Risk Tolerance: {user_info.get('risk_tolerance', 'moderate')}

Use the available tools to calculate retirement needs and recommend asset allocation.
Provide a detailed retirement plan summary."""

        tools_to_use = self.tools
        llm_with_tools = self.llm.bind_tools(tools_to_use)

        response = llm_with_tools.invoke([HumanMessage(content=prompt)])

        # Execute tool calls if any
        if hasattr(response, 'tool_calls') and response.tool_calls:
            tool_results = []
            for tool_call in response.tool_calls:
                tool_name = tool_call["name"]
                tool_args = tool_call["args"]

                # Find and execute the tool
                for tool in tools_to_use:
                    if tool.name == tool_name:
                        result = tool.invoke(tool_args)
                        tool_results.append(result)

            # Create final summary with tool results
            summary_prompt = f"""Based on these calculations:
{chr(10).join(tool_results)}

Create a comprehensive retirement planning summary with specific recommendations."""

            final_response = self.llm.invoke([HumanMessage(content=summary_prompt)])
            summary = final_response.content
        else:
            summary = response.content

        state["plan_summaries"]["Retirement Planning"] = summary
        state["messages"].append(AIMessage(content=f"✓ Retirement Planning Complete"))

        return state

class InsuranceAgent:
    def __init__(self, llm):
        self.llm = llm
        self.tools = [calculate_life_insurance]

    def process(self, state: AgentState) -> AgentState:
        user_info = state["user_info"]

        prompt = f"""You are an Insurance Planning Specialist. Analyze the user's situation and recommend appropriate insurance coverage.

User Information:
- Age: {user_info.get('age', 'Not provided')}
- Annual Income: ${user_info.get('annual_income', 0):,.2f}
- Number of Dependents: {user_info.get('num_dependents', 0)}
- Current Savings: ${user_info.get('savings', 0):,.2f}
- Outstanding Debts: ${user_info.get('debts', 0):,.2f}

Use the life insurance calculation tool and provide comprehensive insurance recommendations
including life, health, disability, and liability insurance."""

        llm_with_tools = self.llm.bind_tools(self.tools)
        response = llm_with_tools.invoke([HumanMessage(content=prompt)])

        if hasattr(response, 'tool_calls') and response.tool_calls:
            tool_results = []
            for tool_call in response.tool_calls:
                result = calculate_life_insurance.invoke(tool_call["args"])
                tool_results.append(result)

            summary_prompt = f"""Based on these calculations:
{chr(10).join(tool_results)}

Create a comprehensive insurance planning summary covering life, health, disability, and liability insurance."""

            final_response = self.llm.invoke([HumanMessage(content=summary_prompt)])
            summary = final_response.content
        else:
            summary = response.content

        state["plan_summaries"]["Insurance Planning"] = summary
        state["messages"].append(AIMessage(content=f"✓ Insurance Planning Complete"))

        return state

class EstateAgent:
    def __init__(self, llm):
        self.llm = llm
        self.tools = [calculate_estate_tax, calculate_education_fund]

    def process(self, state: AgentState) -> AgentState:
        user_info = state["user_info"]

        prompt = f"""You are an Estate Planning Specialist. Create a comprehensive estate plan.

User Information:
- Age: {user_info.get('age', 'Not provided')}
- Total Assets: ${user_info.get('total_assets', user_info.get('savings', 0)):,.2f}
- Number of Children: {user_info.get('num_children', 0)}
- Children Ages: {user_info.get('children_ages', [])}

Use the available tools to calculate estate taxes and education funding needs.
Provide recommendations for wills, trusts, beneficiaries, and legacy planning."""

        llm_with_tools = self.llm.bind_tools(self.tools)
        response = llm_with_tools.invoke([HumanMessage(content=prompt)])

        if hasattr(response, 'tool_calls') and response.tool_calls:
            tool_results = []
            for tool_call in response.tool_calls:
                tool_name = tool_call["name"]
                for tool in self.tools:
                    if tool.name == tool_name:
                        result = tool.invoke(tool_call["args"])
                        tool_results.append(result)

            summary_prompt = f"""Based on these calculations:
{chr(10).join(tool_results)}

Create a comprehensive estate planning summary including wills, trusts, beneficiary designations,
education funding, and tax minimization strategies."""

            final_response = self.llm.invoke([HumanMessage(content=summary_prompt)])
            summary = final_response.content
        else:
            summary = response.content

        state["plan_summaries"]["Estate Planning"] = summary
        state["messages"].append(AIMessage(content=f"✓ Estate Planning Complete"))

        return state

class WealthAgent:
    def __init__(self, llm):
        self.llm = llm
        self.tools = [calculate_wealth_allocation]

    def process(self, state: AgentState) -> AgentState:
        user_info = state["user_info"]

        prompt = f"""You are a Personal Wealth Management Specialist. Create a comprehensive wealth management strategy.

User Information:
- Age: {user_info.get('age', 'Not provided')}
- Annual Income: ${user_info.get('annual_income', 0):,.2f}
- Total Assets: ${user_info.get('total_assets', user_info.get('savings', 0)):,.2f}
- Risk Tolerance: {user_info.get('risk_tolerance', 'moderate')}

Use the asset allocation tool and provide recommendations for:
- Investment strategy
- Tax optimization
- Cash flow management
- Diversification
- Regular portfolio review schedule"""

        llm_with_tools = self.llm.bind_tools(self.tools)
        response = llm_with_tools.invoke([HumanMessage(content=prompt)])

        if hasattr(response, 'tool_calls') and response.tool_calls:
            tool_results = []
            for tool_call in response.tool_calls:
                result = calculate_wealth_allocation.invoke(tool_call["args"])
                tool_results.append(result)

            summary_prompt = f"""Based on these calculations:
{chr(10).join(tool_results)}

Create a comprehensive wealth management summary including investment strategy, tax optimization,
diversification recommendations, and monitoring schedule."""

            final_response = self.llm.invoke([HumanMessage(content=summary_prompt)])
            summary = final_response.content
        else:
            summary = response.content

        state["plan_summaries"]["Personal Wealth Management"] = summary
        state["messages"].append(AIMessage(content=f"✓ Wealth Management Planning Complete"))

        return state

# STEP 5: Orchestrator Agent
The code block in Step 5 defines the OrchestratorAgent class, which is responsible for coordinating the execution of the specialized financial planning agents and generating an integrated summary.

Here's a breakdown of its components:

* Initialization (__init__): The OrchestratorAgent is initialized with the llm (Large Language Model) and instances of each specialized agent (RetirementAgent, InsuranceAgent, EstateAgent, WealthAgent). This allows it to call upon these individual agents to perform their specific tasks.

* route method: This method takes the current AgentState as input and orchestrates the workflow. It iterates through the selected_plans in the AgentState and, for each selected plan that hasn't been summarized yet, it calls the process method of the corresponding specialized agent. After all relevant individual plans have been processed, it then calls the create_integrated_summary method to consolidate the results.

* create_integrated_summary method: This method is responsible for taking all the individual plan summaries generated by the specialized agents, along with the user_info, and using the LLM to synthesize them into a single, cohesive "Executive Summary." This summary provides an overview, highlights key recommendations, identifies synergies, and proposes an action plan.

In [ ]:
class OrchestratorAgent:
    def __init__(self, llm):
        self.llm = llm
        self.retirement_agent = RetirementAgent(llm)
        self.insurance_agent = InsuranceAgent(llm)
        self.estate_agent = EstateAgent(llm)
        self.wealth_agent = WealthAgent(llm)

    def route(self, state: AgentState) -> AgentState:
        """Route to appropriate agents based on selected plans"""
        selected = state["selected_plans"]

        if "Retirement Planning" in selected and "Retirement Planning" not in state["plan_summaries"]:
            state = self.retirement_agent.process(state)

        if "Insurance Planning" in selected and "Insurance Planning" not in state["plan_summaries"]:
            state = self.insurance_agent.process(state)

        if "Estate Planning" in selected and "Estate Planning" not in state["plan_summaries"]:
            state = self.estate_agent.process(state)

        if "Personal Wealth Management" in selected and "Personal Wealth Management" not in state["plan_summaries"]:
            state = self.wealth_agent.process(state)

        # Generate integrated summary
        state = self.create_integrated_summary(state)

        return state

    def create_integrated_summary(self, state: AgentState) -> AgentState:
        """Create an integrated summary of all plans"""
        summaries = state["plan_summaries"]
        user_info = state["user_info"]

        prompt = f"""You are a Senior Financial Advisor. Create an integrated Executive Summary
that combines all the individual plan summaries into a cohesive financial plan.

User Profile:
{json.dumps(user_info, indent=2)}

Individual Plan Summaries:
{chr(10).join([f"### {plan}:{chr(10)}{summary}{chr(10)}" for plan, summary in summaries.items()])}

Create an Executive Summary that:
1. Provides an overview of the client's financial situation
2. Highlights key recommendations from each planning area
3. Identifies synergies and priorities across plans
4. Provides a clear action plan with timeline
5. Notes any areas requiring immediate attention"""

        response = self.llm.invoke([HumanMessage(content=prompt)])
        state["plan_summaries"]["Executive Summary"] = response.content

        return state

# STEP 6: Main Application
The code block in Step 6 defines the main application logic for the financial plan summary, primarily through two functions: collect_user_info and run_planning_application.

* collect_user_info(selected_plans: List[str]) -> Dict[str, Any]:

  * Purpose: This function is responsible for interactively gathering specific financial and personal details from the user based on the planning areas they selected. It ensures that only relevant information (e.g., retirement age if 'Retirement Planning' is chosen) is requested.
  * Process: It prompts the user for inputs like age, annual income, savings, and then conditionally asks for more specific details related to chosen plans (e.g., desired retirement age, number of dependents, children's ages, risk tolerance).
  * Output: It returns a dictionary (info) containing all the collected user data.

* run_planning_application():

  * Purpose: This is the central function that orchestrates the entire financial planning process from start to finish.
  * Process:
    1. Plan Selection: It first presents the user with a list of available financial planning options (Retirement, Insurance, Estate, Wealth Management) and allows them to select which plans they want.
    2. User Information Collection: It then calls collect_user_info to gather the necessary data from the user based on their selections.
    3. State Initialization: An initial AgentState is created, populated with the collected user_info and selected_plans.
    4. Orchestration: It instantiates the OrchestratorAgent and calls its route method, passing the initial state. This triggers the specialized agents to generate their respective plan summaries and the integrated executive summary.
    5. Display Results: Finally, it prints out the comprehensive financial plan, starting with the Executive Summary and then presenting each individual plan summary.

In [ ]:
def collect_user_info(selected_plans: List[str]) -> Dict[str, Any]:
    """Collect user information based on selected plans"""
    info = {}

    print("\n" + "="*60)
    print("PLEASE PROVIDE YOUR INFORMATION")
    print("="*60)

    # Basic info needed for all plans
    info['age'] = int(input("Your current age: "))
    info['annual_income'] = float(input("Annual income ($): "))
    info['savings'] = float(input("Current savings/assets ($): "))

    if "Retirement Planning" in selected_plans:
        info['retirement_age'] = int(input("Desired retirement age: "))
        risk = input("Risk tolerance (conservative/moderate/aggressive): ").lower()
        info['risk_tolerance'] = risk if risk in ['conservative', 'moderate', 'aggressive'] else 'moderate'

    if "Insurance Planning" in selected_plans:
        info['num_dependents'] = int(input("Number of dependents: "))
        info['debts'] = float(input("Outstanding debts ($): "))

    if "Estate Planning" in selected_plans:
        info['num_children'] = int(input("Number of children: "))
        if info['num_children'] > 0:
            ages = input(f"Ages of children (comma-separated): ")
            info['children_ages'] = [int(age.strip()) for age in ages.split(',')]
        else:
            info['children_ages'] = []
        info['total_assets'] = info['savings']

    if "Personal Wealth Management" in selected_plans:
        info['total_assets'] = info['savings']
        if 'risk_tolerance' not in info:
            risk = input("Risk tolerance (conservative/moderate/aggressive): ").lower()
            info['risk_tolerance'] = risk if risk in ['conservative', 'moderate', 'aggressive'] else 'moderate'

    return info

def run_planning_application():
    """Main application runner"""
    print("\n" + "="*60)
    print("FINANCIAL PLAN SUMMARY APPLICATION")
    print("="*60)

    # Plan selection
    available_plans = [
        "Retirement Planning",
        "Insurance Planning",
        "Estate Planning",
        "Personal Wealth Management"
    ]

    print("\nAvailable Planning Options:")
    for i, plan in enumerate(available_plans, 1):
        print(f"{i}. {plan}")

    selection = input("\nEnter plan numbers (comma-separated, e.g., 1,3,4): ")
    selected_indices = [int(x.strip()) - 1 for x in selection.split(',')]
    selected_plans = [available_plans[i] for i in selected_indices if 0 <= i < len(available_plans)]

    if not selected_plans:
        print("No valid plans selected. Exiting.")
        return None

    print(f"\nSelected Plans: {', '.join(selected_plans)}")

    # Collect user information
    user_info = collect_user_info(selected_plans)

    # Initialize state
    initial_state = AgentState(
        messages=[],
        user_info=user_info,
        selected_plans=selected_plans,
        plan_summaries={},
        next_agent=""
    )

    # Run orchestrator
    print("\n" + "="*60)
    print("GENERATING YOUR FINANCIAL PLAN...")
    print("="*60 + "\n")

    orchestrator = OrchestratorAgent(llm)
    final_state = orchestrator.route(initial_state)

    # Display results
    print("\n" + "="*60)
    print("YOUR COMPREHENSIVE FINANCIAL PLAN")
    print("="*60 + "\n")

    # Show executive summary first
    if "Executive Summary" in final_state["plan_summaries"]:
        print("EXECUTIVE SUMMARY")
        print("-" * 60)
        print(final_state["plan_summaries"]["Executive Summary"])
        print("\n")

    # Show individual plans
    for plan, summary in final_state["plan_summaries"].items():
        if plan != "Executive Summary":
            print(f"\n{plan.upper()}")
            print("-" * 60)
            print(summary)
            print("\n")

    return final_state

# STEP 7: Interactive Chat for Follow-up Questions

The code block in Step 7 defines the chat_interface function, which provides an interactive chat for follow-up questions after the financial plan has been generated.

Here's a breakdown of its components and functionality:

* chat_interface(planning_state: AgentState) function:
  * Purpose: This function enables a conversational interaction with the user, allowing them to ask questions about their generated financial plan. The AI assistant uses the context of the complete financial plan to provide relevant and specific answers.
  * Input: It takes the planning_state (the final AgentState after the financial plan has been created) as input. This state contains all the user_info and plan_summaries.
  * Initialization: If planning_state is None (meaning no plan was generated), it exits. Otherwise, it sets up an introductory message and an empty conversation_history list to store previous exchanges.
  * Context Creation: It constructs a SystemMessage that serves as the AI's internal context. This message includes the user_info and all Financial Plan Summaries from the planning_state. This ensures the AI always has the complete financial picture when responding.
  * Interactive Loop: It enters a while True loop to allow continuous interaction:
    1. User Input: It prompts the user for a question. If the user types 'exit', 'quit', or 'bye', the chat ends.
    2. Message Construction: It builds a list of messages for the LLM, starting with the SystemMessage context, followed by the conversation_history, and finally the current HumanMessage from the user.
    3. LLM Invocation: It calls the llm.invoke() method with the constructed list of messages to get a response from the language model.
    4. History Update: The user's input and the AI's response are appended to the conversation_history to maintain conversational memory.
    5. Context Management: It keeps the conversation_history length manageable (e.g., last 10 messages) to prevent context window overflow and maintain efficiency.
    6. Display Response: The AI's response is printed to the console.

In [ ]:
def chat_interface(planning_state: AgentState):
    """Interactive chat interface for follow-up questions"""
    if planning_state is None:
        print("No planning session found. Please run the planning application first.")
        return

    print("\n" + "="*60)
    print("INTERACTIVE CHAT - Ask follow-up questions about your plan")
    print("Type 'exit' to quit")
    print("="*60 + "\n")

    conversation_history = []

    # Create context from planning state
    context = f"""You are a financial advisor assistant. You have access to the following information
about the client and their financial plan:

User Information:
{json.dumps(planning_state['user_info'], indent=2)}

Financial Plan Summaries:
{chr(10).join([f"### {plan}:{chr(10)}{summary}{chr(10)}" for plan, summary in planning_state['plan_summaries'].items()])}

Answer the user's questions based on this context. Be helpful, specific, and reference
the plans when relevant."""

    while True:
        user_input = input("\nYou: ").strip()

        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Thank you for using the Financial Planning Assistant!")
            break

        if not user_input:
            continue

        # Build conversation with context
        messages = [SystemMessage(content=context)]
        messages.extend(conversation_history)
        messages.append(HumanMessage(content=user_input))

        # Get response
        response = llm.invoke(messages)

        # Update conversation history
        conversation_history.append(HumanMessage(content=user_input))
        conversation_history.append(AIMessage(content=response.content))

        # Keep only last 10 messages to manage context
        if len(conversation_history) > 10:
            conversation_history = conversation_history[-10:]

        print(f"\nAssistant: {response.content}")

# RUN THE APPLICATION

In [ ]:
# Execute the planning application
print("Starting Financial Plan Summary Application...\n")
planning_result = run_planning_application()

# # Start chat interface if planning was successful
# if planning_result:
#     print("\n" + "="*60)
#     print("Planning complete! You can now ask follow-up questions.")
#     print("="*60)
#     chat_interface(planning_result)

In [ ]:
!pip install plotly numpy

# STEP 8: Visualization Functions

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np

def create_retirement_projection_chart(user_info: Dict[str, Any]):
    """Create retirement savings projection chart"""
    current_age = user_info.get('age', 30)
    retirement_age = user_info.get('retirement_age', 65)
    current_savings = user_info.get('savings', 0)
    annual_income = user_info.get('annual_income', 0)

    # Calculate monthly contribution needed
    years_to_retirement = retirement_age - current_age
    annual_expenses = annual_income * 0.8  # Assume 80% of income needed in retirement

    # Project savings with different contribution rates
    ages = list(range(current_age, 86))

    # Scenario 1: Current savings only (no additional contributions)
    no_contrib = []
    # Scenario 2: Conservative (5% of income)
    conservative = []
    # Scenario 3: Moderate (10% of income)
    moderate = []
    # Scenario 4: Aggressive (15% of income)
    aggressive = []

    for i, age in enumerate(ages):
        years_elapsed = i
        growth_rate = 0.07  # 7% annual return

        # No contributions
        no_contrib.append(current_savings * ((1 + growth_rate) ** years_elapsed))

        # With contributions (future value of annuity formula)
        if age < retirement_age:
            conservative_contrib = annual_income * 0.05
            moderate_contrib = annual_income * 0.10
            aggressive_contrib = annual_income * 0.15

            fv_contrib_c = conservative_contrib * (((1 + growth_rate) ** years_elapsed - 1) / growth_rate)
            fv_contrib_m = moderate_contrib * (((1 + growth_rate) ** years_elapsed - 1) / growth_rate)
            fv_contrib_a = aggressive_contrib * (((1 + growth_rate) ** years_elapsed - 1) / growth_rate)

            conservative.append(current_savings * ((1 + growth_rate) ** years_elapsed) + fv_contrib_c)
            moderate.append(current_savings * ((1 + growth_rate) ** years_elapsed) + fv_contrib_m)
            aggressive.append(current_savings * ((1 + growth_rate) ** years_elapsed) + fv_contrib_a)
        else:
            # After retirement, start drawing down
            withdrawal_rate = 0.04
            years_in_retirement = age - retirement_age

            conservative.append(max(0, conservative[-1] * ((1 + growth_rate - withdrawal_rate) ** 1)))
            moderate.append(max(0, moderate[-1] * ((1 + growth_rate - withdrawal_rate) ** 1)))
            aggressive.append(max(0, aggressive[-1] * ((1 + growth_rate - withdrawal_rate) ** 1)))

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=ages, y=no_contrib, name='No Additional Savings',
                             line=dict(color='red', width=2, dash='dash')))
    fig.add_trace(go.Scatter(x=ages, y=conservative, name='Conservative (5% savings)',
                             line=dict(color='orange', width=2)))
    fig.add_trace(go.Scatter(x=ages, y=moderate, name='Moderate (10% savings)',
                             line=dict(color='blue', width=3)))
    fig.add_trace(go.Scatter(x=ages, y=aggressive, name='Aggressive (15% savings)',
                             line=dict(color='green', width=2)))

    # Add retirement age line
    fig.add_vline(x=retirement_age, line_dash="dot", line_color="gray",
                  annotation_text="Retirement Age", annotation_position="top")

    fig.update_layout(
        title='Retirement Savings Projection',
        xaxis_title='Age',
        yaxis_title='Portfolio Value ($)',
        hovermode='x unified',
        template='plotly_white',
        height=500,
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
        yaxis=dict(tickformat='$,.0f')
    )

    return fig

def create_asset_allocation_pie(user_info: Dict[str, Any]):
    """Create asset allocation pie chart"""
    age = user_info.get('age', 30)
    risk_tolerance = user_info.get('risk_tolerance', 'moderate')
    total_assets = user_info.get('total_assets', user_info.get('savings', 0))

    # Calculate allocation
    base_stock_pct = 100 - age

    if risk_tolerance.lower() == "aggressive":
        stock_pct = min(90, base_stock_pct + 10)
    elif risk_tolerance.lower() == "conservative":
        stock_pct = max(20, base_stock_pct - 20)
    else:
        stock_pct = base_stock_pct

    bond_pct = 100 - stock_pct

    # Further breakdown
    allocations = {
        'US Stocks': stock_pct * 0.6,
        'International Stocks': stock_pct * 0.3,
        'Emerging Markets': stock_pct * 0.1,
        'Bonds': bond_pct * 0.7,
        'Cash/Money Market': bond_pct * 0.3
    }

    values = [total_assets * (pct / 100) for pct in allocations.values()]

    fig = go.Figure(data=[go.Pie(
        labels=list(allocations.keys()),
        values=values,
        hole=.3,
        marker=dict(colors=['#2E86AB', '#A23B72', '#F18F01', '#06A77D', '#D4AF37'])
    )])

    fig.update_layout(
        title=f'Recommended Asset Allocation ({risk_tolerance.title()} Profile)',
        annotations=[dict(text=f'${total_assets:,.0f}', x=0.5, y=0.5, font_size=20, showarrow=False)],
        height=500
    )

    return fig

def create_insurance_coverage_chart(user_info: Dict[str, Any]):
    """Create insurance coverage comparison chart"""
    annual_income = user_info.get('annual_income', 0)
    num_dependents = user_info.get('num_dependents', 0)
    debts = user_info.get('debts', 0)
    savings = user_info.get('savings', 0)

    # Calculate recommended coverage
    base_coverage = (annual_income * 10) + debts - savings
    dependent_factor = 1 + (num_dependents * 0.2)
    recommended_coverage = base_coverage * dependent_factor

    # Different insurance types
    insurance_types = ['Life Insurance', 'Disability Insurance', 'Critical Illness', 'Long-term Care']
    recommended = [
        recommended_coverage,
        annual_income * 5,  # Disability
        annual_income * 3,  # Critical illness
        300000  # Long-term care average
    ]

    current = [
        recommended_coverage * 0.4,  # Assume underinsured
        annual_income * 2,
        0,
        0
    ]

    fig = go.Figure()

    fig.add_trace(go.Bar(
        name='Current Coverage',
        x=insurance_types,
        y=current,
        marker_color='lightcoral'
    ))

    fig.add_trace(go.Bar(
        name='Recommended Coverage',
        x=insurance_types,
        y=recommended,
        marker_color='lightseagreen'
    ))

    fig.update_layout(
        title='Insurance Coverage Analysis',
        xaxis_title='Insurance Type',
        yaxis_title='Coverage Amount ($)',
        barmode='group',
        template='plotly_white',
        height=500,
        yaxis=dict(tickformat='$,.0f')
    )

    return fig

def create_education_funding_chart(user_info: Dict[str, Any]):
    """Create education funding progress chart"""
    num_children = user_info.get('num_children', 0)
    children_ages = user_info.get('children_ages', [])

    if num_children == 0 or not children_ages:
        return None

    cost_per_year = 30000
    inflation_rate = 0.05

    children_data = []
    for i, age in enumerate(children_ages):
        years_until_college = max(0, 18 - age)
        future_cost = cost_per_year * ((1 + inflation_rate) ** years_until_college)
        total_needed = future_cost * 4

        # Assume some savings already (30% of needed)
        current_savings = total_needed * 0.3

        children_data.append({
            'child': f'Child {i+1} (Age {age})',
            'needed': total_needed,
            'saved': current_savings,
            'gap': total_needed - current_savings,
            'years': years_until_college
        })

    fig = go.Figure()

    children_labels = [c['child'] for c in children_data]

    fig.add_trace(go.Bar(
        name='Amount Saved',
        x=children_labels,
        y=[c['saved'] for c in children_data],
        marker_color='mediumseagreen'
    ))

    fig.add_trace(go.Bar(
        name='Funding Gap',
        x=children_labels,
        y=[c['gap'] for c in children_data],
        marker_color='tomato'
    ))

    fig.update_layout(
        title='Education Funding Status',
        xaxis_title='Child',
        yaxis_title='Amount ($)',
        barmode='stack',
        template='plotly_white',
        height=500,
        yaxis=dict(tickformat='$,.0f')
    )

    return fig

def create_net_worth_projection(user_info: Dict[str, Any]):
    """Create net worth projection over time"""
    current_age = user_info.get('age', 30)
    annual_income = user_info.get('annual_income', 0)
    current_savings = user_info.get('savings', 0)
    debts = user_info.get('debts', 0)
    retirement_age = user_info.get('retirement_age', 65)

    ages = list(range(current_age, retirement_age + 30))

    assets = []
    liabilities = []
    net_worth = []

    annual_savings = annual_income * 0.15
    debt_payment = debts * 0.1 if debts > 0 else 0

    current_asset = current_savings
    current_debt = debts

    for i, age in enumerate(ages):
        growth_rate = 0.07

        if age < retirement_age:
            # Accumulation phase
            current_asset = current_asset * (1 + growth_rate) + annual_savings
            current_debt = max(0, current_debt - debt_payment)
        else:
            # Retirement phase
            current_asset = current_asset * (1 + growth_rate) - (annual_income * 0.8 * 0.04)
            current_debt = 0

        assets.append(current_asset)
        liabilities.append(current_debt)
        net_worth.append(current_asset - current_debt)

    fig = make_subplots(specs=[[{"secondary_y": False}]])

    fig.add_trace(go.Scatter(
        x=ages, y=assets, name='Assets',
        fill='tonexty',
        line=dict(color='green', width=2)
    ))

    fig.add_trace(go.Scatter(
        x=ages, y=liabilities, name='Liabilities',
        fill='tozeroy',
        line=dict(color='red', width=2)
    ))

    fig.add_trace(go.Scatter(
        x=ages, y=net_worth, name='Net Worth',
        line=dict(color='blue', width=3, dash='dash')
    ))

    fig.add_vline(x=retirement_age, line_dash="dot", line_color="gray",
                  annotation_text="Retirement", annotation_position="top")

    fig.update_layout(
        title='Net Worth Projection Over Time',
        xaxis_title='Age',
        yaxis_title='Amount ($)',
        hovermode='x unified',
        template='plotly_white',
        height=500,
        yaxis=dict(tickformat='$,.0f')
    )

    return fig

def create_monthly_budget_breakdown(user_info: Dict[str, Any]):
    """Create monthly budget breakdown"""
    annual_income = user_info.get('annual_income', 0)
    monthly_income = annual_income / 12

    # 50/30/20 rule with adjustments
    categories = {
        'Housing': monthly_income * 0.28,
        'Transportation': monthly_income * 0.15,
        'Food': monthly_income * 0.12,
        'Utilities': monthly_income * 0.08,
        'Insurance': monthly_income * 0.07,
        'Savings/Investments': monthly_income * 0.15,
        'Entertainment': monthly_income * 0.08,
        'Personal Care': monthly_income * 0.04,
        'Other': monthly_income * 0.03
    }

    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8',
              '#6C5CE7', '#FDCB6E', '#E17055', '#A29BFE']

    fig = go.Figure(data=[go.Bar(
        y=list(categories.keys()),
        x=list(categories.values()),
        orientation='h',
        marker=dict(color=colors),
        text=[f'${v:,.0f}' for v in categories.values()],
        textposition='auto',
    )])

    fig.update_layout(
        title=f'Recommended Monthly Budget (Total: ${monthly_income:,.0f})',
        xaxis_title='Monthly Amount ($)',
        yaxis_title='Category',
        template='plotly_white',
        height=500,
        showlegend=False,
        xaxis=dict(tickformat='$,.0f')
    )

    return fig

def display_all_visualizations(planning_state: AgentState):
    """Display all relevant visualizations based on selected plans"""
    if planning_state is None:
        print("No planning data available. Run the planning application first.")
        return

    user_info = planning_state['user_info']
    selected_plans = planning_state['selected_plans']

    print("\n" + "="*60)
    print("FINANCIAL PLAN VISUALIZATIONS")
    print("="*60 + "\n")

    # Net Worth Projection (always show)
    print("📊 Generating Net Worth Projection...")
    fig = create_net_worth_projection(user_info)
    fig.show()

    # Retirement Planning visualizations
    if "Retirement Planning" in selected_plans:
        print("\n📈 Generating Retirement Savings Projection...")
        fig = create_retirement_projection_chart(user_info)
        fig.show()

    # Wealth Management visualizations
    if "Personal Wealth Management" in selected_plans or "Retirement Planning" in selected_plans:
        print("\n🥧 Generating Asset Allocation Chart...")
        fig = create_asset_allocation_pie(user_info)
        fig.show()

        print("\n💰 Generating Monthly Budget Breakdown...")
        fig = create_monthly_budget_breakdown(user_info)
        fig.show()

    # Insurance Planning visualizations
    if "Insurance Planning" in selected_plans:
        print("\n🛡️ Generating Insurance Coverage Analysis...")
        fig = create_insurance_coverage_chart(user_info)
        fig.show()

    # Estate Planning visualizations
    if "Estate Planning" in selected_plans:
        num_children = user_info.get('num_children', 0)
        if num_children > 0:
            print("\n🎓 Generating Education Funding Status...")
            fig = create_education_funding_chart(user_info)
            if fig:
                fig.show()

    print("\n" + "="*60)
    print("All visualizations generated successfully!")
    print("="*60)

## Generate and display all visualizations

In [ ]:
display_all_visualizations(planning_result)

## Start interactive chat for follow-up questions

In [ ]:
chat_interface(planning_result)